## 欢迎进入 ModelWhale Notebook  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [3]:
# 查看个人持久化工作区文件
!ls /home/mw/project/

output


In [6]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

quant4533


In [4]:
# 一、读取数据集
import pandas as pd
import os

# 1. 定义文件路径
detail_path = "/home/mw/input/quant4533/ruc_Class25Q1_details.csv"
rent_path = "/home/mw/input/quant4533/ruc_Class25Q1_rent.csv"
test_path = "/home/mw/input/quant4533/ruc_Class25Q1_test.csv"
train_path = "/home/mw/input/quant4533/ruc_Class25Q1_train.csv"

# 2. 读取数据（确保原始数据无乱码）
try:
    detail_df = pd.read_csv(detail_path)
    rent_df = pd.read_csv(rent_path)
    test_df = pd.read_csv(test_path)
    train_df = pd.read_csv(train_path)
    
    print("✅ 数据读取成功！")
    print(f"训练集形状: {train_df.shape}")
    print(f"测试集形状: {test_df.shape}")
    print(f"详情数据形状: {detail_df.shape}")
    print(f"租金数据形状: {rent_df.shape}")
    
except Exception as e:
    print(f"❌ 读取数据时出错: {e}")

# 3. 创建输出目录
output_dir = "/home/mw/project/output"
os.makedirs(output_dir, exist_ok=True)

# 4. 重新保存为 UTF-8 with BOM 编码（解决 Excel 乱码问题）
try:
    detail_df.to_csv(f"{output_dir}/details.csv", index=False, encoding="utf_8_sig")
    rent_df.to_csv(f"{output_dir}/rent.csv", index=False, encoding="utf_8_sig")
    test_df.to_csv(f"{output_dir}/test.csv", index=False, encoding="utf_8_sig")
    train_df.to_csv(f"{output_dir}/train.csv", index=False, encoding="utf_8_sig")
    
    print(f"💾 数据已保存到: {output_dir}")
    print("编码格式: utf_8_sig（兼容 Excel 中文）")
    
except Exception as e:
    print(f"❌ 保存数据时出错: {e}")

# 5. 验证文件是否存在
print("\n检查保存的文件:")
saved_files = os.listdir(output_dir)
for file in saved_files:
    print(f"- {file}")

# 6. 显示前 3 行数据（验证无乱码）
print("\n训练集前 3 行（检查中文显示）:")
display(train_df.head(3))

✅ 数据读取成功！
训练集形状: (84133, 31)
测试集形状: (14786, 31)
详情数据形状: (3100, 27)
租金数据形状: (84150, 22)
💾 数据已保存到: /home/mw/project/output
编码格式: utf_8_sig（兼容 Excel 中文）

检查保存的文件:
- onehot_encoder.joblib
- test.csv
- train.csv
- details.csv
- train_ready.csv
- rent.csv
- train_processed.csv
- train_ready_fixed.csv

训练集前 3 行（检查中文显示）:


城市    区域     板块    环线   小区名称        价格      房屋户型       所在楼层     建筑面积  \
0   0  79.0  111.0  二至三环  人定湖西里   6564200  2室1厅1厨1卫  中楼层 (共5层)    52.3㎡   
1   0  43.0  231.0  五至六环  龙跃苑四区   4174000  3室1厅1厨1卫   顶层 (共6层)  127.44㎡   
2   0  97.0   54.0  五至六环    名都园  16310000  4室2厅1厨4卫   底层 (共3层)  228.54㎡   

     套内面积  ... 产权所属 抵押信息      房屋优势                  核心卖点  \
0     NaN  ...  非共有  NaN  装修、房本满五年  此房是南北通透小板楼，户型方正，格局合理   
1  123.7㎡  ...  非共有  NaN  装修、房本满五年        南北通透商品房自住装修无个税   
2     NaN  ...  非共有  NaN     房本满五年         有挑空客厅私家车库花园和*   

                                                户型介绍  \
0                     房子是南北通透户型方正采光好，前后没有遮挡视野好，通风效果好   
1  房子三居一卫，户型方正，南北通透，客厅朝南带阳台，主卧朝南，东向有窗户，次卧、厨房朝北，厨房...   
2               地上三层地下一层四期密度低没有公寓一层车库可以做卧室花园有100平米左右   

                                                周边配套 交通出行         lon  \
0   医院、公园、超市，生活便利，火箭军医院、积水潭医院，双秀公园，人定湖公园，物美超市、世纪华联等。  NaN  116.389326   
1  医院：北京京都儿童医院、昌平中西结合医院，积水潭医院配套设施：美廉美，工商，农行，邮政等银行...  NaN  116.354287   
2                                                NaN  NaN  116.543168   

         lat      年份  
0  39.963727  2018.0  
1  40.079237  2017.0  
2  40.078165  2018.0  

[3 rows x 31 columns]

In [6]:
# 二、数据预处理
# 2.11 test_环线
def preprocess_ring(df):
    """
    环线列预处理函数（适用于train和test数据集）
    处理逻辑：
    1. 填补空白值为"外环外"（最普遍情况）
    2. 统一不同表达方式
    3. 有序编码（数值越大表示离市中心越远）
    """
    # 复制数据避免修改原DataFrame
    df = df.copy()
    
    # 统一表达方式
    ring_mapping = {
        '内环内': '一环内',
        '二环内': '一环至二环',
        '内环至中环': '一环至二环',
        '内环至外环': '一环至外环',
        '中环至外环': '二环至外环',
        '一至二环': '一环至二环'  # 新增映射
    }
    
    # 填补空白
    df['环线'] = df['环线'].fillna('外环外')
    
    # 统一表达
    df['环线'] = df['环线'].replace(ring_mapping)
    
    # 有序编码字典（数值越大离市中心越远）
    ring_order = {
        '一环内': 1,
        '一环至二环': 2,
        '二至三环': 3,
        '三至四环': 4,
        '四至五环': 5,
        '五至六环': 6,
        '三环外': 4.5,
        '四环外': 5.5,
        '六环外': 7,
        '一环至外环': 3.5,
        '二环至外环': 4,
        '外环外': 6.5
    }
    
    # 应用编码
    df['环线_编码'] = df['环线'].map(ring_order)
    
    # 添加是否核心区标志
    df['是否核心区'] = df['环线_编码'].apply(lambda x: 1 if x <= 3 else 0)
    
    # 返回处理后的数据和编码字典
    return df, ring_order

# ==================== 测试验证 ==================== 
# 在训练集上应用
train_df, ring_order = preprocess_ring(train_df)

# 验证处理结果
print("===== 环线值分布 =====")
print(train_df['环线'].value_counts(dropna=False))

print("\n===== 编码后分布 =====")
print(train_df.groupby('环线')['环线_编码'].first())

print("\n===== 样本示例 =====")
print(train_df[['环线', '环线_编码', '是否核心区']].head(10))

# 保存编码映射供测试集使用
ring_encoding_map = {
    '原始值': list(ring_order.keys()),
    '编码值': list(ring_order.values())
}
pd.DataFrame(ring_encoding_map).to_csv(f"{output_dir}/ring_encoding_map.csv", index=False)
    

===== 环线值分布 =====
外环外      43445
一环内       9323
二至三环      8994
五至六环      5573
三至四环      4453
一环至二环     3359
四至五环      2785
三环外       2197
六环外       1927
一环至外环     1044
二环至外环      923
四环外        110
Name: 环线, dtype: int64

===== 编码后分布 =====
环线
一环内      1.0
一环至二环    2.0
一环至外环    3.5
三环外      4.5
三至四环     4.0
二环至外环    4.0
二至三环     3.0
五至六环     6.0
六环外      7.0
四环外      5.5
四至五环     5.0
外环外      6.5
Name: 环线_编码, dtype: float64

===== 样本示例 =====
     环线  环线_编码  是否核心区
0  二至三环    3.0      1
1  五至六环    6.0      0
2  五至六环    6.0      0
3  三至四环    4.0      0
4  三至四环    4.0      0
5  三至四环    4.0      0
6  五至六环    6.0      0
7  三至四环    4.0      0
8  四至五环    5.0      0
9  三至四环    4.0      0


In [7]:
print(train_df['环线_编码'].isnull().sum())

0


In [11]:
# 2.12测试集环线列预处理
def preprocess_test_ring(test_df, encoding_map_path=f"{output_dir}/ring_encoding_map.csv"):
    """
    测试集环线列预处理（使用训练集生成的编码规则）
    参数:
        test_df: 测试集DataFrame
        encoding_map_path: 训练集保存的编码映射文件路径
    """
    # 加载训练集生成的编码映射
    try:
        ring_encoding = pd.read_csv(encoding_map_path)
        ring_order = dict(zip(ring_encoding['原始值'], ring_encoding['编码值']))
    except Exception as e:
        print(f"❌ 加载编码映射失败: {e}")
        return test_df
    
    # 复制数据避免修改原DataFrame
    test_df = test_df.copy()
    
    # 统一表达方式（与训练集相同）
    ring_mapping = {
        '内环内': '一环内',
        '二环内': '一环至二环',
        '内环至中环': '一环至二环',
        '内环至外环': '一环至外环',
        '中环至外环': '二环至外环',
        '一至二环': '一环至二环'
    }
    
    # 填补空白（与训练集相同）
    test_df['环线'] = test_df['环线'].fillna('外环外')
    
    # 统一表达
    test_df['环线'] = test_df['环线'].replace(ring_mapping)
    
    # 应用训练集的编码规则
    test_df['环线_编码'] = test_df['环线'].map(ring_order)
    
    # 添加是否核心区标志（与训练集相同阈值）
    test_df['是否核心区'] = test_df['环线_编码'].apply(lambda x: 1 if x <= 3 else 0)
    
    # 检查是否有新出现的环线值
    unseen_rings = set(test_df['环线'].unique()) - set(ring_order.keys())
    if unseen_rings:
        print(f"⚠️ 发现训练集未出现的环线值: {unseen_rings}")
        print("将设为默认值'外环外'的编码")
        test_df['环线_编码'] = test_df['环线_编码'].fillna(ring_order['外环外'])
    
    return test_df

# 应用预处理
test_df = preprocess_test_ring(test_df)

# 验证处理结果
print("\n===== 测试集环线处理验证 =====")
print("1. 编码后缺失值数量:", test_df['环线_编码'].isnull().sum())
print("2. 环线值分布:")
print(test_df['环线'].value_counts(dropna=False))
print("\n3. 样本示例:")
print(test_df[['环线', '环线_编码', '是否核心区']].head(10))

# 保存处理后的测试集
test_df.to_csv(f"{output_dir}/test_preprocessed.csv", index=False, encoding="utf_8_sig")
print(f"\n💾 预处理后的测试集已保存到: {output_dir}/test_preprocessed.csv")


===== 测试集环线处理验证 =====
1. 编码后缺失值数量: 0
2. 环线值分布:
外环外      6826
二至三环     1273
一环至二环    1254
一环内      1235
五至六环     1184
三至四环     1128
二环至外环     607
六环外       543
四至五环      403
三环外       189
一环至外环      84
四环外        60
Name: 环线, dtype: int64

3. 样本示例:
     环线  环线_编码  是否核心区
0  四至五环    5.0      0
1  四至五环    5.0      0
2  五至六环    6.0      0
3   外环外    6.5      0
4  二至三环    3.0      1
5   外环外    6.5      0
6  四至五环    5.0      0
7  五至六环    6.0      0
8   外环外    6.5      0
9   外环外    6.5      0

💾 预处理后的测试集已保存到: /home/mw/project/output/test_preprocessed.csv


In [15]:
# 2.2 “小区名称”
# 删除小区名称列的标准化处理
def remove_neighborhood_col(df, col_name='小区名称'):
    """
    删除指定列并记录操作日志
    参数:
        df: 输入DataFrame
        col_name: 要删除的列名
    返回:
        处理后的DataFrame
    """
    if col_name in df.columns:
        print(f"✅ 正在删除列: {col_name}")
        df = df.drop(columns=[col_name])
        # 记录删除操作
        with open(f"{output_dir}/data_preprocessing_log.txt", 'a') as f:
            f.write(f"[{pd.Timestamp.now()}] 删除列: {col_name}\n")
    else:
        print(f"⚠️ 列 {col_name} 不存在，跳过删除")
    return df

# 应用删除操作
print("===== 训练集处理 =====")
train_df = remove_neighborhood_col(train_df)

print("\n===== 测试集处理 =====")
test_df = remove_neighborhood_col(test_df)

# 验证删除结果
print("\n===== 处理结果验证 =====")
print("训练集剩余列:", train_df.columns.tolist())
print("测试集剩余列:", test_df.columns.tolist())

# 保存处理后的数据
train_df.to_csv(f"{output_dir}/train_no_neighborhood.csv", index=False, encoding='utf_8_sig')
test_df.to_csv(f"{output_dir}/test_no_neighborhood.csv", index=False, encoding='utf_8_sig')

print(f"\n💾 处理后的数据已保存:")
print(f"- {output_dir}/train_no_neighborhood.csv")
print(f"- {output_dir}/test_no_neighborhood.csv")
print(f"- 操作日志: {output_dir}/data_preprocessing_log.txt")

===== 训练集处理 =====
⚠️ 列 小区名称 不存在，跳过删除

===== 测试集处理 =====
⚠️ 列 小区名称 不存在，跳过删除

===== 处理结果验证 =====
训练集剩余列: ['城市', '区域', '板块', '环线', '价格', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '环线_编码', '是否核心区']
测试集剩余列: ['ID', '城市', '区域', '板块', '环线', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '环线_编码', '是否核心区', '卧室数', '客厅数', '厨房数', '卫生间数', '总房间数', '卧室卫生间比', '是否标准户型', '是否零居室', '是否豪宅户型']

💾 处理后的数据已保存:
- /home/mw/project/output/train_no_neighborhood.csv
- /home/mw/project/output/test_no_neighborhood.csv
- 操作日志: /home/mw/project/output/data_preprocessing_log.txt


In [18]:
# 2.31房屋户型test



import os
import pandas as pd
from functools import reduce

# 确保所有目录存在
output_dir = "/home/mw/project/output"
columns_dir = os.path.join(output_dir, "columns")
os.makedirs(columns_dir, exist_ok=True)  # 关键修复：创建columns子目录

def preprocess_house_layout(df, is_train=True):
    """房屋户型列预处理函数（与之前保持一致）"""
    df = df.copy()
    df['房屋户型'] = df['房屋户型'].str.replace('房间', '室').str.replace('卫室', '卫 室')
    df['房屋户型'] = df['房屋户型'].fillna('1室1厅1厨1卫')
    
    pattern = r'(\d+)室(\d*)厅(\d*)厨(\d*)卫'
    extracted = df['房屋户型'].str.extract(pattern)
    
    result = pd.DataFrame()
    result['卧室数'] = extracted[0].fillna('0').astype(int)
    result['客厅数'] = extracted[1].fillna('0').astype(int)
    result['厨房数'] = extracted[2].fillna('0').astype(int)
    result['卫生间数'] = extracted[3].fillna('0').astype(int)
    
    result.loc[result['卧室数'] > 10, '卧室数'] = 10
    result.loc[result['卫生间数'] > result['卧室数'] + 3, '卫生间数'] = result['卧室数'] + 3
    
    result['总房间数'] = result['卧室数'] + result['客厅数']
    result['卧室卫生间比'] = result['卧室数'] / (result['卫生间数'] + 0.01)
    result['是否标准户型'] = ((result['卧室数'] >= 1) & 
                          (result['客厅数'] >= 1) & 
                          (result['厨房数'] >= 1) & 
                          (result['卫生间数'] >= 1)).astype(int)
    result['是否零居室'] = (result['卧室数'] == 0).astype(int)
    result['是否豪宅户型'] = (result['卧室数'] >= 4).astype(int)
    
    result = result.add_prefix('户型_')
    return result, df['房屋户型']

# ==================== 训练集处理 ====================
try:
    train_path = os.path.join(output_dir, "train_stage1_preprocessed.csv")
    train_df = pd.read_csv(train_path)
    
    # 应用户型预处理
    train_layout_features, train_original_layout = preprocess_house_layout(train_df)
    
    # 保存户型特征（修正路径）
    train_layout_path = os.path.join(columns_dir, "train_房屋户型.csv")
    train_layout_features.to_csv(train_layout_path, index=False, encoding='utf_8_sig')
    print(f"✅ 户型特征已保存到: {train_layout_path}")

    # 更新合并文件
    def update_merged_file():
        merged_train = pd.read_csv(os.path.join(output_dir, "train_stage1_preprocessed.csv"))
        layout_features = pd.read_csv(train_layout_path)
        final_train = pd.concat([merged_train, layout_features], axis=1)
        final_path = os.path.join(output_dir, "train_merged.csv")
        final_train.to_csv(final_path, index=False, encoding='utf_8_sig')
        print(f"✅ 合并后的训练集已保存: {final_path}")
        return final_train

    final_train_df = update_merged_file()
    
    # 验证
    print("\n===== 处理结果样例 =====")
    display(final_train_df[['房屋户型', '户型_卧室数', '户型_卫生间数']].head(3))
    
except Exception as e:
    print(f"❌ 处理过程中出错: {e}")
    print("请检查以下路径是否存在:")
    print(f"- 输入文件: {os.path.join(output_dir, 'train_stage1_preprocessed.csv')}")
    print(f"- 输出目录: {columns_dir}")

✅ 户型特征已保存到: /home/mw/project/output/columns/train_房屋户型.csv
✅ 合并后的训练集已保存: /home/mw/project/output/train_merged.csv

===== 处理结果样例 =====


房屋户型  户型_卧室数  户型_卫生间数
0  2室1厅1厨1卫       2        1
1  3室1厅1厨1卫       3        1
2  4室2厅1厨4卫       4        4

In [19]:
# 2.32 房屋户型test
# 确保测试集输入文件存在
test_input_path = f"{output_dir}/test_stage1_preprocessed.csv"
if not os.path.exists(test_input_path):
    raise FileNotFoundError(f"测试集输入文件不存在: {test_input_path}")

# 加载测试集基础数据（已处理环线和小区名称）
test_df = pd.read_csv(test_input_path)

# 应用相同的户型预处理函数
test_layout_features, test_original_layout = preprocess_house_layout(test_df, is_train=False)

# 验证处理一致性
print("===== 测试集户型处理验证 =====")
print("1. 特征维度检查 - 训练集 vs 测试集")
print(f"  训练集特征: {train_layout_features.shape[1]}列, 测试集特征: {test_layout_features.shape[1]}列")

print("\n2. 特征名称一致性检查:")
print("  差异列:", set(train_layout_features.columns) - set(test_layout_features.columns))

print("\n3. 数值范围检查:")
print("  测试集卧室数分布:\n", test_layout_features['户型_卧室数'].value_counts().sort_index())

# 保存测试集户型特征
test_layout_path = f"{output_dir}/columns/test_房屋户型.csv"
test_layout_features.to_csv(test_layout_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集户型特征已保存到: {test_layout_path}")

# 更新测试集合并文件
def update_test_merged():
    """合并测试集所有已处理列"""
    # 加载基础文件
    merged_test = pd.read_csv(f"{output_dir}/test_stage1_preprocessed.csv")
    
    # 加载户型特征
    test_layout = pd.read_csv(test_layout_path)
    
    # 按列合并
    final_test = pd.concat([merged_test, test_layout], axis=1)
    
    # 保存
    final_test_path = f"{output_dir}/test_merged.csv"
    final_test.to_csv(final_test_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_test_path}")
    return final_test

final_test_df = update_test_merged()

# 验证合并结果
print("\n===== 测试集合并结果样例 =====")
sample_cols = ['环线_编码', '户型_卧室数', '户型_是否豪宅户型']
print(final_test_df[sample_cols].head(3))

print("\n===== 当前测试集列结构 =====")
print(final_test_df.columns.tolist())

===== 测试集户型处理验证 =====
1. 特征维度检查 - 训练集 vs 测试集
  训练集特征: 9列, 测试集特征: 9列

2. 特征名称一致性检查:
  差异列: set()

3. 数值范围检查:
  测试集卧室数分布:
 0      43
1    1748
2    6533
3    5318
4     961
5     149
6      23
7       9
8       2
Name: 户型_卧室数, dtype: int64

💾 测试集户型特征已保存到: /home/mw/project/output/columns/test_房屋户型.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged.csv

===== 测试集合并结果样例 =====
   环线_编码  户型_卧室数  户型_是否豪宅户型
0    5.0       3          0
1    5.0       4          1
2    6.0       2          0

===== 当前测试集列结构 =====
['ID', '城市', '区域', '板块', '环线', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '环线_编码', '是否核心区', '户型_卧室数', '户型_客厅数', '户型_厨房数', '户型_卫生间数', '户型_总房间数', '户型_卧室卫生间比', '户型_是否标准户型', '户型_是否零居室', '户型_是否豪宅户型']


In [22]:
def preprocess_floor(df):
    """
    修正版的所在楼层列预处理函数
    """
    df = df.copy()
    
    # 更全面的字符清理
    df['所在楼层'] = df['所在楼层'].str.replace(r'[❑#\-·]', '', regex=True).str.strip()
    
    # 更灵活的正则表达式（匹配中文/英文括号）
    pattern = r'([^\s（(]+)\s*[（(]共\s*(\d+)\s*层[)）]'
    extracted = df['所在楼层'].str.extract(pattern)
    
    # 提取楼层类型（更宽松的处理）
    df['楼层类型'] = extracted[0].str.strip()
    df['总楼层数'] = extracted[1].fillna('0').astype(int)
    
    # 标准化楼层类型表述
    floor_type_mapping = {
        '低楼层': '低楼层', '低层': '低楼层',
        '中楼层': '中楼层', '中层': '中楼层',
        '高楼层': '高楼层', '高层': '高楼层',
        '顶楼': '顶层', '顶楼层': '顶层', '顶层': '顶层',
        '底层': '底层', '底楼': '底层',
        '地下室': '地下室', '地下层': '地下室'
    }
    df['楼层类型'] = df['楼层类型'].map(floor_type_mapping).fillna('中楼层')  # 默认值
    
    # 重新计算相对位置（添加更多业务规则）
    df['相对位置'] = df.apply(lambda x: calculate_relative_position(x['楼层类型'], x['总楼层数']), axis=1)
    
    # 创建衍生特征
    df['是否地下室'] = (df['楼层类型'] == '地下室').astype(int)
    df['是否顶层'] = (df['楼层类型'] == '顶层').astype(int)
    df['是否底层'] = (df['楼层类型'] == '底层').astype(int)
    
    # 更合理的建筑高度分箱
    bins = [-1, 3, 7, 18, 33, 100]  # 调整分箱边界
    labels = ['低层', '多层', '小高层', '高层', '超高层']
    df['建筑高度类型'] = pd.cut(df['总楼层数'], bins=bins, labels=labels)
    
    return df

def calculate_relative_position(floor_type, total_floor):
    """更精确的相对位置计算"""
    position_map = {
        '地下室': 0.0,
        '底层': 0.15 if total_floor > 6 else 0.1,
        '低楼层': 0.25 + min(total_floor/100, 0.1),
        '中楼层': 0.5 + min(total_floor/200, 0.15),
        '高楼层': 0.75 - max((total_floor-30)/100, 0),
        '顶层': 1.0 - (0.05 if total_floor > 30 else 0)
    }
    return position_map.get(floor_type, 0.5)

    # 重新加载原始数据（避免链式处理）
train_df = pd.read_csv("/home/mw/input/quant4533/ruc_Class25Q1_train.csv")

# 应用修正后的预处理
train_floor_processed = preprocess_floor(train_df)

# 验证关键字段
print("===== 修正后验证 =====")
print("1. 成功提取记录占比:", 
      f"{len(train_floor_processed[train_floor_processed['总楼层数']>0])/len(train_floor_processed):.1%}")

print("\n2. 楼层类型分布:")
print(train_floor_processed['楼层类型'].value_counts())

print("\n3. 样例数据:")
sample = train_floor_processed[['所在楼层', '楼层类型', '总楼层数', '相对位置']].head(5)
display(sample)

===== 修正后验证 =====
1. 成功提取记录占比: 99.8%

2. 楼层类型分布:
中楼层    29916
高楼层    25548
低楼层    23706
顶层      2253
底层      2010
地下室      700
Name: 楼层类型, dtype: int64

3. 样例数据:


所在楼层 楼层类型  总楼层数   相对位置
0   中楼层 (共5层)  中楼层     5  0.525
1    顶层 (共6层)   顶层     6  1.000
2    底层 (共3层)   底层     3  0.100
3  低楼层 (共10层)  低楼层    10  0.350
4  中楼层 (共10层)  中楼层    10  0.550

In [23]:
# 加载测试集基础数据
test_merged_path = f"{output_dir}/test_merged.csv"
test_df = pd.read_csv(test_merged_path)

# 应用相同的预处理函数
test_floor_processed = preprocess_floor(test_df)

# 验证处理一致性
print("===== 测试集楼层处理验证 =====")
print("1. 成功提取记录占比:", 
      f"{len(test_floor_processed[test_floor_processed['总楼层数']>0])/len(test_floor_processed):.1%}")

print("\n2. 楼层类型分布:")
print(test_floor_processed['楼层类型'].value_counts())

print("\n3. 衍生特征统计:")
print("是否地下室:", test_floor_processed['是否地下室'].mean())
print("是否顶层:", test_floor_processed['是否顶层'].mean())

# 保存楼层特征（仅新生成的列）
floor_feature_cols = ['楼层类型', '总楼层数', '相对位置', 
                     '是否地下室', '是否顶层', '是否底层', '建筑高度类型']
test_floor_features = test_floor_processed[floor_feature_cols].add_prefix('楼层_')

# 保存到columns目录
test_floor_path = f"{output_dir}/columns/test_所在楼层.csv"
test_floor_features.to_csv(test_floor_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集楼层特征已保存到: {test_floor_path}")

# 更新测试集合并文件
def update_test_merged():
    merged_test = pd.read_csv(test_merged_path)
    floor_features = pd.read_csv(test_floor_path)
    final_test = pd.concat([merged_test, floor_features], axis=1)
    
    final_path = f"{output_dir}/test_merged_v2.csv"
    final_test.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_path}")
    return final_test

final_test_df = update_test_merged()

# 显示处理样例
print("\n===== 测试集处理结果样例 =====")
sample_cols = ['所在楼层', '楼层_楼层类型', '楼层_相对位置', '楼层_建筑高度类型']
display(final_test_df[sample_cols].head(5))

===== 测试集楼层处理验证 =====
1. 成功提取记录占比: 100.0%

2. 楼层类型分布:
中楼层    5908
高楼层    4313
低楼层    3931
底层      322
顶层      306
地下室       6
Name: 楼层类型, dtype: int64

3. 衍生特征统计:
是否地下室: 0.00040578926011091576
是否顶层: 0.020695252265656702

💾 测试集楼层特征已保存到: /home/mw/project/output/columns/test_所在楼层.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged_v2.csv

===== 测试集处理结果样例 =====


所在楼层 楼层_楼层类型  楼层_相对位置 楼层_建筑高度类型
0  中楼层 (共25层)     中楼层    0.625        高层
1   低楼层 (共9层)     低楼层    0.340       小高层
2   中楼层 (共8层)     中楼层    0.540       小高层
3   中楼层 (共6层)     中楼层    0.530        多层
4    底层 (共3层)      底层    0.100        低层

In [24]:
def preprocess_area(df):
    """
    建筑面积和套内面积预处理函数
    处理逻辑：
    1. 清除面积单位符号
    2. 处理缺失值（基于得房率估算）
    3. 创建衍生特征
    4. 异常值处理
    """
    df = df.copy()
    
    # 清除面积单位符号（㎡）并转换为数值
    df['建筑面积'] = df['建筑面积'].str.replace('㎡', '').astype(float)
    df['套内面积'] = df['套内面积'].str.replace('㎡', '').astype(float)
    
    # 计算得房率（套内/建筑）
    df['得房率'] = df['套内面积'] / df['建筑面积']
    
    # 处理缺失的套内面积（用建筑面积*中位数得房率）
    median_ratio = df['得房率'].median()
    df['套内面积'] = df['套内面积'].fillna(df['建筑面积'] * median_ratio)
    
    # 处理异常值
    df.loc[df['建筑面积'] < 15, '建筑面积'] = 15  # 最小合理面积
    df.loc[df['套内面积'] > df['建筑面积'], '套内面积'] = df['建筑面积'] * 0.9  # 套内不应大于建筑面积
    
    # 重新计算得房率
    df['得房率'] = df['套内面积'] / df['建筑面积']
    
    # 创建衍生特征
    df['公摊面积'] = df['建筑面积'] - df['套内面积']
    df['面积等级'] = pd.cut(df['建筑面积'],
                         bins=[0, 50, 90, 144, 200, 300, 1000],
                         labels=['极小', '小', '中', '大', '超大', '豪宅'])
    df['是否豪宅'] = (df['建筑面积'] > 144).astype(int)
    df['是否小户型'] = (df['建筑面积'] < 50).astype(int)
    
    return df

# ==================== 训练集处理 ====================
# 加载之前合并的数据
train_merged_path = f"{output_dir}/train_merged_v2.csv"
train_df = pd.read_csv(train_merged_path)

# 应用面积预处理
train_area_processed = preprocess_area(train_df)

# 验证处理结果
print("===== 面积处理验证 =====")
print("1. 缺失值处理结果:")
print(" - 套内面积缺失值:", train_area_processed['套内面积'].isnull().sum())
print(" - 得房率中位数:", round(train_area_processed['得房率'].median(), 3))

print("\n2. 面积统计描述:")
print(train_area_processed[['建筑面积', '套内面积', '得房率']].describe())

print("\n3. 面积等级分布:")
print(train_area_processed['面积等级'].value_counts())

# 保存面积特征（仅新生成的列）
area_feature_cols = ['建筑面积', '套内面积', '得房率', '公摊面积', 
                    '面积等级', '是否豪宅', '是否小户型']
train_area_features = train_area_processed[area_feature_cols]

# 添加前缀便于后续合并
train_area_features = train_area_features.add_prefix('面积_')

# 保存到columns目录
train_area_path = f"{output_dir}/columns/train_面积.csv"
train_area_features.to_csv(train_area_path, index=False, encoding='utf_8_sig')
print(f"\n💾 面积特征已保存到: {train_area_path}")

# 更新合并文件
def update_train_merged():
    merged_train = pd.read_csv(train_merged_path)
    area_features = pd.read_csv(train_area_path)
    final_train = pd.concat([merged_train, area_features], axis=1)
    
    final_path = f"{output_dir}/train_merged_v3.csv"  # 更新版本号
    final_train.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的训练集已更新: {final_path}")
    return final_train

final_train_df = update_train_merged()

# 显示处理样例
print("\n===== 处理结果样例 =====")
sample_cols = ['建筑面积', '套内面积', '面积_得房率', '面积_面积等级']
display(final_train_df[sample_cols].head(5))

===== 面积处理验证 =====
1. 缺失值处理结果:
 - 套内面积缺失值: 0
 - 得房率中位数: 0.805

2. 面积统计描述:
               建筑面积          套内面积           得房率
count  84133.000000  84133.000000  84133.000000
mean      96.686607     78.215413      0.805819
std       63.325146     52.185640      0.038985
min       15.000000      1.000000      0.011690
25%       66.010000     53.034445      0.805016
50%       88.900000     71.485408      0.805016
75%      116.000000     93.780000      0.805016
max    10337.000000   8321.448949      1.000000

3. 面积等级分布:
小     35325
中     31684
极小     8786
大      5831
超大     1861
豪宅      638
Name: 面积等级, dtype: int64

💾 面积特征已保存到: /home/mw/project/output/columns/train_面积.csv
🔄 合并后的训练集已更新: /home/mw/project/output/train_merged_v3.csv

===== 处理结果样例 =====


建筑面积    套内面积    面积_得房率 面积_面积等级
0    52.3㎡     NaN  0.805016       小
1  127.44㎡  123.7㎡  0.970653       中
2  228.54㎡     NaN  0.805016      超大
3    43.6㎡  29.39㎡  0.674083      极小
4   39.85㎡  29.94㎡  0.751317      极小

In [25]:
# 加载测试集合并数据
test_merged_path = f"{output_dir}/test_merged_v2.csv"
test_df = pd.read_csv(test_merged_path)

# 应用相同的预处理函数（使用训练集计算的得房率中位数）
def preprocess_test_area(df, train_median_ratio):
    """
    测试集面积预处理（保持与训练集一致的处理逻辑）
    参数:
        df: 测试集DataFrame
        train_median_ratio: 训练集计算的得房率中位数
    """
    df = df.copy()
    
    # 统一单位处理
    df['建筑面积'] = df['建筑面积'].str.replace('㎡', '').astype(float)
    df['套内面积'] = df['套内面积'].str.replace('㎡', '').astype(float)
    
    # 使用训练集的得房率中位数填充缺失值（关键一致性保障）
    df['套内面积'] = df['套内面积'].fillna(df['建筑面积'] * train_median_ratio)
    
    # 应用相同的异常值处理阈值
    df.loc[df['建筑面积'] < 15, '建筑面积'] = 15
    df.loc[df['套内面积'] > df['建筑面积'], '套内面积'] = df['建筑面积'] * 0.9
    
    # 计算得房率
    df['得房率'] = df['套内面积'] / df['建筑面积']
    
    # 相同的衍生特征
    df['公摊面积'] = df['建筑面积'] - df['套内面积']
    df['面积等级'] = pd.cut(df['建筑面积'],
                         bins=[0, 50, 90, 144, 200, 300, 1000],
                         labels=['极小', '小', '中', '大', '超大', '豪宅'])
    df['是否豪宅'] = (df['建筑面积'] > 144).astype(int)
    df['是否小户型'] = (df['建筑面积'] < 50).astype(int)
    
    return df

# 获取训练集的得房率中位数（关键步骤！）
train_median_ratio = pd.read_csv(f"{output_dir}/columns/train_面积.csv")['面积_得房率'].median()

# 应用预处理
test_area_processed = preprocess_test_area(test_df, train_median_ratio)

# 验证处理一致性
print("===== 测试集面积处理验证 =====")
print("1. 使用的训练集得房率中位数:", round(train_median_ratio, 3))
print("2. 测试集得房率中位数:", round(test_area_processed['得房率'].median(), 3))

print("\n3. 面积统计描述:")
print(test_area_processed[['建筑面积', '套内面积', '得房率']].describe())

print("\n4. 面积等级分布:")
print(test_area_processed['面积等级'].value_counts())

# 保存面积特征（仅新生成的列）
test_area_features = test_area_processed[[
    '建筑面积', '套内面积', '得房率', '公摊面积', 
    '面积等级', '是否豪宅', '是否小户型'
]].add_prefix('面积_')

# 保存到columns目录
test_area_path = f"{output_dir}/columns/test_面积.csv"
test_area_features.to_csv(test_area_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集面积特征已保存到: {test_area_path}")

# 更新测试集合并文件
def update_test_merged():
    merged_test = pd.read_csv(test_merged_path)
    area_features = pd.read_csv(test_area_path)
    final_test = pd.concat([merged_test, area_features], axis=1)
    
    final_path = f"{output_dir}/test_merged_v3.csv"
    final_test.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_path}")
    return final_test

final_test_df = update_test_merged()

# 显示处理样例
print("\n===== 测试集处理结果样例 =====")
sample_cols = ['建筑面积', '套内面积', '面积_得房率', '面积_是否豪宅']
display(final_test_df[sample_cols].head(5))

===== 测试集面积处理验证 =====
1. 使用的训练集得房率中位数: 0.805
2. 测试集得房率中位数: 0.805

3. 面积统计描述:
               建筑面积          套内面积           得房率
count  14786.000000  14786.000000  14786.000000
mean      92.494791     74.955246      0.808176
std       39.914068     33.215169      0.036972
min       18.760000      1.000000      0.017056
25%       66.780000     53.744871      0.805016
50%       87.985000     70.550000      0.805016
75%      109.980000     88.836363      0.805016
max      594.380000    478.485327      1.000000

4. 面积等级分布:
小     6794
中     5602
极小    1346
大      796
超大     197
豪宅      51
Name: 面积等级, dtype: int64

💾 测试集面积特征已保存到: /home/mw/project/output/columns/test_面积.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged_v3.csv

===== 测试集处理结果样例 =====


建筑面积     套内面积    面积_得房率  面积_是否豪宅
0   209.2㎡  165.71㎡  0.792113        1
1  163.69㎡      NaN  0.805016        1
2  102.92㎡      NaN  0.805016        0
3  109.66㎡      NaN  0.805016        0
4    57.2㎡      NaN  0.805016        0

In [26]:
def preprocess_orientation(df):
    """
    房屋朝向列预处理函数
    处理逻辑：
    1. 标准化方向表述（去除特殊符号、统一格式）
    2. 提取主朝向和组合特征
    3. 创建衍生特征
    """
    df = df.copy()
    
    # 统一格式处理（去除符号、空格标准化）
    df['房屋朝向'] = df['房屋朝向'].str.replace(r'[□\s]', '', regex=True).str.strip()
    
    # 提取基础方向（8个主要方位）
    base_directions = ['东', '南', '西', '北', '东南', '东北', '西南', '西北']
    
    # 创建各方向的二元特征
    for direction in base_directions:
        df[f'朝向_{direction}'] = df['房屋朝向'].apply(lambda x: 1 if direction in x else 0)
    
    # 判断是否多朝向组合
    df['朝向_组合数'] = df['房屋朝向'].apply(lambda x: len([d for d in base_directions if d in x]))
    
    # 提取主朝向（按价值排序：南 > 东南 > 东 > 西南 > 北 > 西 > 东北 > 西北）
    priority_order = ['南', '东南', '东', '西南', '北', '西', '东北', '西北']
    def get_main_direction(x):
        for direction in priority_order:
            if direction in x:
                return direction
        return '其他'
    df['朝向_主方向'] = df['房屋朝向'].apply(get_main_direction)
    
    # 创建优质朝向标志
    df['朝向_优质'] = df['朝向_主方向'].isin(['南', '东南', '东']).astype(int)
    
    # 特殊组合特征
    df['朝向_南北通透'] = (df['朝向_南'] & df['朝向_北']).astype(int)
    df['朝向_东西通透'] = (df['朝向_东'] & df['朝向_西']).astype(int)
    
    return df

# ==================== 训练集处理 ====================
# 加载之前合并的数据
train_merged_path = f"{output_dir}/train_merged_v3.csv"
train_df = pd.read_csv(train_merged_path)

# 应用朝向预处理
train_orientation_processed = preprocess_orientation(train_df)

# 验证处理结果
print("===== 朝向处理验证 =====")
print("1. 主方向分布:")
print(train_orientation_processed['朝向_主方向'].value_counts())

print("\n2. 组合数分布:")
print(train_orientation_processed['朝向_组合数'].value_counts().sort_index())

print("\n3. 优质朝向占比:", 
      f"{train_orientation_processed['朝向_优质'].mean():.1%}")

# 保存朝向特征（仅新生成的列）
orientation_cols = [col for col in train_orientation_processed.columns 
                   if col.startswith('朝向_')]
train_orientation_features = train_orientation_processed[orientation_cols]

# 保存到columns目录
train_orientation_path = f"{output_dir}/columns/train_房屋朝向.csv"
train_orientation_features.to_csv(train_orientation_path, index=False, encoding='utf_8_sig')
print(f"\n💾 朝向特征已保存到: {train_orientation_path}")

# 更新合并文件
def update_train_merged():
    merged_train = pd.read_csv(train_merged_path)
    orientation_features = pd.read_csv(train_orientation_path)
    final_train = pd.concat([merged_train, orientation_features], axis=1)
    
    final_path = f"{output_dir}/train_merged_v4.csv"
    final_train.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的训练集已更新: {final_path}")
    return final_train

final_train_df = update_train_merged()

# 显示处理样例
print("\n===== 处理结果样例 =====")
sample_cols = ['房屋朝向', '朝向_主方向', '朝向_组合数', '朝向_优质', '朝向_南北通透']
display(final_train_df[sample_cols].head(8))

===== 朝向处理验证 =====
1. 主方向分布:
南    69288
东     8051
北     4567
西     2227
Name: 朝向_主方向, dtype: int64

2. 组合数分布:
1    34225
2    30713
3    17297
4     1459
5      178
6      257
7        4
Name: 朝向_组合数, dtype: int64

3. 优质朝向占比: 91.9%

💾 朝向特征已保存到: /home/mw/project/output/columns/train_房屋朝向.csv
🔄 合并后的训练集已更新: /home/mw/project/output/train_merged_v4.csv

===== 处理结果样例 =====


房屋朝向 朝向_主方向  朝向_组合数  朝向_优质  朝向_南北通透
0  南 北      南       2      1        1
1  南 北      南       2      1        1
2    南      南       1      1        0
3    北      北       1      0        0
4    南      南       1      1        0
5    南      南       1      1        0
6  南 北      南       2      1        1
7   东南      南       3      1        0

In [27]:
# 查看多朝向组合样本
multi_orientation = final_train_df[final_train_df['朝向_组合数'] > 2]
print(multi_orientation[['房屋朝向', '朝向_主方向', '朝向_组合数']])

          房屋朝向 朝向_主方向  朝向_组合数
7           东南      南       3
21          西南      南       3
28        东南 西      南       4
39       东 南 北      南       4
40     东 南 西 北      南       6
...        ...    ...     ...
84092       东南      南       3
84101       西南      南       3
84108       东南      南       3
84125      西 北      北       3
84129       东南      南       3

[19195 rows x 3 columns]


In [28]:
# test的朝向处理
# 加载测试集合并数据
test_merged_path = f"{output_dir}/test_merged_v3.csv"
test_df = pd.read_csv(test_merged_path)

# 应用相同的预处理函数
test_orientation_processed = preprocess_orientation(test_df)

# 验证处理一致性
print("===== 测试集朝向处理验证 =====")
print("1. 主方向分布:")
print(test_orientation_processed['朝向_主方向'].value_counts())

print("\n2. 组合数分布:")
print(test_orientation_processed['朝向_组合数'].value_counts().sort_index())

print("\n3. 优质朝向占比:", 
      f"{test_orientation_processed['朝向_优质'].mean():.1%}")

# 保存朝向特征（仅新生成的列）
orientation_cols = [col for col in test_orientation_processed.columns 
                   if col.startswith('朝向_')]
test_orientation_features = test_orientation_processed[orientation_cols]

# 保存到columns目录
test_orientation_path = f"{output_dir}/columns/test_房屋朝向.csv"
test_orientation_features.to_csv(test_orientation_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集朝向特征已保存到: {test_orientation_path}")

# 更新测试集合并文件
def update_test_merged():
    merged_test = pd.read_csv(test_merged_path)
    orientation_features = pd.read_csv(test_orientation_path)
    final_test = pd.concat([merged_test, orientation_features], axis=1)
    
    final_path = f"{output_dir}/test_merged_v4.csv"
    final_test.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_path}")
    return final_test

final_test_df = update_test_merged()

# 显示处理样例
print("\n===== 测试集处理结果样例 =====")
sample_cols = ['房屋朝向', '朝向_主方向', '朝向_组合数', '朝向_优质', '朝向_南北通透']
display(final_test_df[sample_cols].head(8))

===== 测试集朝向处理验证 =====
1. 主方向分布:
南    13153
东      953
北      450
西      230
Name: 朝向_主方向, dtype: int64

2. 组合数分布:
1    6480
2    5821
3    2160
4     278
5      21
6      25
7       1
Name: 朝向_组合数, dtype: int64

3. 优质朝向占比: 95.4%

💾 测试集朝向特征已保存到: /home/mw/project/output/columns/test_房屋朝向.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged_v4.csv

===== 测试集处理结果样例 =====


房屋朝向 朝向_主方向  朝向_组合数  朝向_优质  朝向_南北通透
0  南 北      南       2      1        1
1  南 北      南       2      1        1
2    西      西       1      0        0
3  南 北      南       2      1        1
4  南 北      南       2      1        1
5    南      南       1      1        0
6   东南      南       3      1        0
7    西      西       1      0        0

In [29]:
#train的建筑结构
def preprocess_structure(df):
    """
    建筑结构列预处理函数
    处理逻辑：
    1. 填补空白值为最常见类型（砖混结构）
    2. 有序编码（按结构质量从高到低）
    3. 创建衍生特征
    """
    df = df.copy()
    
    # 填补空白值
    df['建筑结构'] = df['建筑结构'].fillna('砖混结构')
    
    # 统一表述
    df['建筑结构'] = df['建筑结构'].str.replace('钢混结构', '钢筋混凝土结构')
    
    # 有序编码字典（数值越大表示结构质量越好）
    structure_order = {
        '钢结构': 4,           # 最高级
        '钢筋混凝土结构': 3,   # 现代主流
        '框架结构': 2,         # 常见小高层
        '混合结构': 1,         # 过渡类型
        '砖混结构': 0,         # 老旧住宅
        '砖木结构': -1,        # 价值最低
        '未知结构': 0          # 默认按砖混处理
    }
    
    # 应用编码
    df['结构_编码'] = df['建筑结构'].map(structure_order)
    
    # 创建衍生特征
    df['结构_现代建筑'] = (df['结构_编码'] >= 2).astype(int)  # 钢/钢筋混凝土/框架
    df['结构_需警惕'] = (df['结构_编码'] <= -1).astype(int)   # 砖木结构
    
    # 保存编码映射
    pd.DataFrame({
        '原始值': list(structure_order.keys()),
        '编码值': list(structure_order.values())
    }).to_csv(f"{output_dir}/structure_encoding_map.csv", index=False)
    
    return df

# ==================== 训练集处理 ====================
# 加载之前合并的数据
train_merged_path = f"{output_dir}/train_merged_v4.csv"
train_df = pd.read_csv(train_merged_path)

# 应用结构预处理
train_structure_processed = preprocess_structure(train_df)

# 验证处理结果
print("===== 建筑结构处理验证 =====")
print("1. 结构类型分布:")
print(train_structure_processed['建筑结构'].value_counts())

print("\n2. 编码值分布:")
print(train_structure_processed['结构_编码'].value_counts().sort_index())

print("\n3. 衍生特征统计:")
print("现代建筑占比:", f"{train_structure_processed['结构_现代建筑'].mean():.1%}")
print("需警惕结构占比:", f"{train_structure_processed['结构_需警惕'].mean():.1%}")

# 保存结构特征（仅新生成的列）
structure_cols = ['结构_编码', '结构_现代建筑', '结构_需警惕']
train_structure_features = train_structure_processed[structure_cols]

# 保存到columns目录
train_structure_path = f"{output_dir}/columns/train_建筑结构.csv"
train_structure_features.to_csv(train_structure_path, index=False, encoding='utf_8_sig')
print(f"\n💾 结构特征已保存到: {train_structure_path}")

# 更新合并文件
def update_train_merged():
    merged_train = pd.read_csv(train_merged_path)
    structure_features = pd.read_csv(train_structure_path)
    final_train = pd.concat([merged_train, structure_features], axis=1)
    
    final_path = f"{output_dir}/train_merged_v5.csv"
    final_train.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的训练集已更新: {final_path}")
    return final_train

final_train_df = update_train_merged()

# 显示处理样例
print("\n===== 处理结果样例 =====")
sample_cols = ['建筑结构', '结构_编码', '结构_现代建筑']
display(final_train_df[sample_cols].head(5))

===== 建筑结构处理验证 =====
1. 结构类型分布:
钢筋混凝土结构    65198
混合结构        9405
砖混结构        4324
框架结构        2462
未知结构        1715
钢结构          975
砖木结构          54
Name: 建筑结构, dtype: int64

2. 编码值分布:
-1       54
 0     6039
 1     9405
 2     2462
 3    65198
 4      975
Name: 结构_编码, dtype: int64

3. 衍生特征统计:
现代建筑占比: 81.6%
需警惕结构占比: 0.1%

💾 结构特征已保存到: /home/mw/project/output/columns/train_建筑结构.csv
🔄 合并后的训练集已更新: /home/mw/project/output/train_merged_v5.csv

===== 处理结果样例 =====


建筑结构  结构_编码  结构_现代建筑
0  混合结构      1        0
1  混合结构      1        0
2  混合结构      1        0
3  钢混结构      3        1
4  钢混结构      3        1

In [30]:
# 加载测试集合并数据
test_merged_path = f"{output_dir}/test_merged_v4.csv"
test_df = pd.read_csv(test_merged_path)

# 应用相同的预处理（使用训练集生成的编码映射）
def preprocess_test_structure(df):
    """
    测试集建筑结构预处理（保持与训练集一致）
    参数:
        df: 测试集DataFrame
    返回:
        处理后的DataFrame（仅包含结构相关特征）
    """
    # 加载训练集创建的编码映射
    structure_map = pd.read_csv(f"{output_dir}/structure_encoding_map.csv")
    structure_order = dict(zip(structure_map['原始值'], structure_map['编码值']))
    
    df = df.copy()
    df['建筑结构'] = df['建筑结构'].fillna('砖混结构').str.replace('钢混结构', '钢筋混凝土结构')
    
    # 应用编码（未知值设为砖混结构的编码0）
    df['结构_编码'] = df['建筑结构'].map(structure_order).fillna(0)
    
    # 相同的衍生特征
    df['结构_现代建筑'] = (df['结构_编码'] >= 2).astype(int)
    df['结构_需警惕'] = (df['结构_编码'] <= -1).astype(int)
    
    return df[['结构_编码', '结构_现代建筑', '结构_需警惕']]

# 应用预处理
test_structure_features = preprocess_test_structure(test_df)

# 验证处理一致性
print("===== 测试集结构处理验证 =====")
print("1. 编码值分布:")
print(test_structure_features['结构_编码'].value_counts().sort_index())

print("\n2. 衍生特征统计:")
print("现代建筑占比:", f"{test_structure_features['结构_现代建筑'].mean():.1%}")
print("需警惕结构占比:", f"{test_structure_features['结构_需警惕'].mean():.1%}")

# 保存到columns目录
test_structure_path = f"{output_dir}/columns/test_建筑结构.csv"
test_structure_features.to_csv(test_structure_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集结构特征已保存到: {test_structure_path}")

# 更新测试集合并文件
def update_test_merged():
    merged_test = pd.read_csv(test_merged_path)
    structure_features = pd.read_csv(test_structure_path)
    final_test = pd.concat([merged_test, structure_features], axis=1)
    
    final_path = f"{output_dir}/test_merged_v5.csv"
    final_test.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_path}")
    return final_test

final_test_df = update_test_merged()

# 显示处理样例
print("\n===== 测试集处理结果样例 =====")
sample_cols = ['建筑结构', '结构_编码', '结构_现代建筑']
display(final_test_df[sample_cols].head(5))

===== 测试集结构处理验证 =====
1. 编码值分布:
-1        7
 0     1720
 1     1875
 2      310
 3    10794
 4       80
Name: 结构_编码, dtype: int64

2. 衍生特征统计:
现代建筑占比: 75.6%
需警惕结构占比: 0.0%

💾 测试集结构特征已保存到: /home/mw/project/output/columns/test_建筑结构.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged_v5.csv

===== 测试集处理结果样例 =====


建筑结构  结构_编码  结构_现代建筑
0  钢混结构      3        1
1  钢混结构      3        1
2  钢混结构      3        1
3  混合结构      1        0
4  混合结构      1        0

In [31]:
def preprocess_renovation(df):
    """
    装修情况列预处理函数
    处理逻辑：
    1. 填补空白值为最常见类型（简装）
    2. 有序编码（按装修质量从高到低）
    3. 创建衍生特征
    """
    df = df.copy()
    
    # 填补空白值
    df['装修情况'] = df['装修情况'].fillna('简装')
    
    # 有序编码字典（数值越大表示装修越好）
    renovation_order = {
        '精装': 3,    # 最高标准
        '简装': 1,     # 基础装修
        '毛坯': 0,     # 无装修
        '其他': 1      # 默认按简装处理
    }
    
    # 应用编码
    df['装修_编码'] = df['装修情况'].map(renovation_order)
    
    # 创建衍生特征
    df['装修_精装修'] = (df['装修_编码'] == 3).astype(int)
    df['装修_需装修'] = (df['装修_编码'] == 0).astype(int)
    
    # 保存编码映射
    pd.DataFrame({
        '原始值': list(renovation_order.keys()),
        '编码值': list(renovation_order.values())
    }).to_csv(f"{output_dir}/renovation_encoding_map.csv", index=False)
    
    return df

# ==================== 训练集处理 ====================
# 加载之前合并的数据
train_merged_path = f"{output_dir}/train_merged_v5.csv"
train_df = pd.read_csv(train_merged_path)

# 应用装修预处理
train_renovation_processed = preprocess_renovation(train_df)

# 验证处理结果
print("===== 装修情况处理验证 =====")
print("1. 装修类型分布:")
print(train_renovation_processed['装修情况'].value_counts())

print("\n2. 编码值分布:")
print(train_renovation_processed['装修_编码'].value_counts().sort_index())

print("\n3. 衍生特征统计:")
print("精装修占比:", f"{train_renovation_processed['装修_精装修'].mean():.1%}")
print("需装修占比:", f"{train_renovation_processed['装修_需装修'].mean():.1%}")

# 保存装修特征（仅新生成的列）
renovation_cols = ['装修_编码', '装修_精装修', '装修_需装修']
train_renovation_features = train_renovation_processed[renovation_cols]

# 保存到columns目录
train_renovation_path = f"{output_dir}/columns/train_装修情况.csv"
train_renovation_features.to_csv(train_renovation_path, index=False, encoding='utf_8_sig')
print(f"\n💾 装修特征已保存到: {train_renovation_path}")

# 更新合并文件
def update_train_merged():
    merged_train = pd.read_csv(train_merged_path)
    renovation_features = pd.read_csv(train_renovation_path)
    final_train = pd.concat([merged_train, renovation_features], axis=1)
    
    final_path = f"{output_dir}/train_merged_v6.csv"
    final_train.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的训练集已更新: {final_path}")
    return final_train

final_train_df = update_train_merged()

# 显示处理样例
print("\n===== 处理结果样例 =====")
sample_cols = ['装修情况', '装修_编码', '装修_精装修']
display(final_train_df[sample_cols].head(5))

===== 装修情况处理验证 =====
1. 装修类型分布:
精装    30529
其他    26186
简装    17948
毛坯     9470
Name: 装修情况, dtype: int64

2. 编码值分布:
0     9470
1    44134
3    30529
Name: 装修_编码, dtype: int64

3. 衍生特征统计:
精装修占比: 36.3%
需装修占比: 11.3%

💾 装修特征已保存到: /home/mw/project/output/columns/train_装修情况.csv
🔄 合并后的训练集已更新: /home/mw/project/output/train_merged_v6.csv

===== 处理结果样例 =====


装修情况  装修_编码  装修_精装修
0   精装      3       1
1   精装      3       1
2   精装      3       1
3   精装      3       1
4   精装      3       1

In [34]:
import pandas as pd
import os

# 确保输出目录存在
output_dir = "/home/mw/project/output"
os.makedirs(output_dir, exist_ok=True)

# 加载测试集合并数据
test_merged_path = f"{output_dir}/test_merged_v5.csv"
test_df = pd.read_csv(test_merged_path)

# 加载训练集创建的编码映射
renovation_map = pd.read_csv(f"{output_dir}/renovation_encoding_map.csv")
renovation_order = dict(zip(renovation_map['原始值'], renovation_map['编码值']))

def preprocess_test_renovation(df, renovation_order):
    """
    测试集装修情况预处理（保持与训练集一致）
    参数:
        df: 测试集DataFrame
        renovation_order: 装修类型编码字典
    返回:
        处理后的DataFrame（仅包含装修相关特征）
    """
    df = df.copy()
    df['装修情况'] = df['装修情况'].fillna('简装')  # 与训练集相同填充
    
    # 应用编码（未知值按简装处理）
    df['装修_编码'] = df['装修情况'].map(renovation_order).fillna(1)  # 简装=1
    
    # 相同的衍生特征
    df['装修_精装修'] = (df['装修_编码'] == 3).astype(int)
    df['装修_需装修'] = (df['装修_编码'] == 0).astype(int)
    
    return df[['装修_编码', '装修_精装修', '装修_需装修']]

# 应用预处理
test_renovation_features = preprocess_test_renovation(test_df, renovation_order)

# 验证处理一致性
print("===== 测试集装修处理验证 =====")
print("1. 编码值分布:")
print(test_renovation_features['装修_编码'].value_counts().sort_index())

print("\n2. 衍生特征统计:")
print("精装修占比:", f"{test_renovation_features['装修_精装修'].mean():.1%}")
print("需装修占比:", f"{test_renovation_features['装修_需装修'].mean():.1%}")

# 检查是否有训练集未出现的装修类型
unique_test_renovation = set(test_df['装修情况'].unique()) - set(renovation_order.keys())
print("\n3. 测试集特有装修类型:", unique_test_renovation if unique_test_renovation else "无")

# 保存到columns目录
test_renovation_path = f"{output_dir}/columns/test_装修情况.csv"
test_renovation_features.to_csv(test_renovation_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集装修特征已保存到: {test_renovation_path}")

# 更新测试集合并文件
def update_test_merged():
    merged_test = pd.read_csv(test_merged_path)
    renovation_features = pd.read_csv(test_renovation_path)
    final_test = pd.concat([merged_test, renovation_features], axis=1)
    
    final_path = f"{output_dir}/test_merged_v6.csv"
    final_test.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_path}")
    return final_test

final_test_df = update_test_merged()

# 显示处理样例
print("\n===== 测试集处理结果样例 =====")
sample_cols = ['装修情况', '装修_编码', '装修_精装修']
display(final_test_df[sample_cols].head(5))

===== 测试集装修处理验证 =====
1. 编码值分布:
0    1430
1    6274
3    7082
Name: 装修_编码, dtype: int64

2. 衍生特征统计:
精装修占比: 47.9%
需装修占比: 9.7%

3. 测试集特有装修类型: {nan}

💾 测试集装修特征已保存到: /home/mw/project/output/columns/test_装修情况.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged_v6.csv

===== 测试集处理结果样例 =====


装修情况  装修_编码  装修_精装修
0   精装      3       1
1   精装      3       1
2   简装      1       0
3   其他      1       0
4   简装      1       0

In [36]:
# 修正后的"梯户比例"列预处理完整代码

import pandas as pd
import os

# 确保输出目录存在
output_dir = "/home/mw/project/output"
os.makedirs(output_dir, exist_ok=True)

def preprocess_elevator_ratio(df):
    """
    梯户比例列预处理函数（修正版）
    处理逻辑：
    1. 提取电梯数量和每梯户数
    2. 计算电梯服务密度
    3. 创建衍生特征（确保列名正确）
    """
    df = df.copy()
    
    # 统一格式处理（去除特殊符号）
    df['梯户比例'] = df['梯户比例'].str.replace(r'[□\s]', '', regex=True)
    
    # 正则表达式提取数值
    pattern = r'(\d+)梯(\d+)户'
    extracted = df['梯户比例'].str.extract(pattern)
    
    # 转换为数值型
    df['电梯数量'] = extracted[0].fillna('1').astype(int)  # 默认1梯
    df['每梯户数'] = extracted[1].fillna('1').astype(int)  # 默认1户
    
    # 计算总户数和服务密度
    df['总户数'] = df['电梯数量'] * df['每梯户数']
    df['电梯服务密度'] = df['总户数'] / df['电梯数量']
    
    # 异常值处理（基于业务逻辑）
    df.loc[df['电梯数量'] > 20, '电梯数量'] = 20
    df.loc[df['每梯户数'] > 100, '每梯户数'] = 100
    
    # 创建衍生特征（修正列名前缀）
    df['拥挤度'] = pd.cut(df['电梯服务密度'],
                       bins=[0, 4, 8, 12, 20, 1000],
                       labels=['非常宽松', '舒适', '一般', '拥挤', '非常拥挤'])
    df['豪宅配置'] = (df['电梯服务密度'] <= 4).astype(int)
    df['经济型'] = (df['电梯服务密度'] >= 12).astype(int)
    
    return df

# ==================== 训练集处理 ====================
# 加载之前合并的数据
train_merged_path = f"{output_dir}/train_merged_v6.csv"
train_df = pd.read_csv(train_merged_path)

# 应用梯户比例预处理
train_elevator_processed = preprocess_elevator_ratio(train_df)

# 验证处理结果
print("===== 梯户比例处理验证 =====")
print("1. 电梯数量分布:")
print(train_elevator_processed['电梯数量'].value_counts().sort_index())

print("\n2. 每梯户数描述统计:")
print(train_elevator_processed['每梯户数'].describe())

print("\n3. 拥挤度分布:")
print(train_elevator_processed['拥挤度'].value_counts())

# 保存梯户特征（修正列名前缀逻辑）
elevator_cols = ['电梯数量', '每梯户数', '总户数', '电梯服务密度', '拥挤度', '豪宅配置', '经济型']
train_elevator_features = train_elevator_processed[elevator_cols]

# 添加前缀（确保与后续引用一致）
train_elevator_features.columns = ['梯户_' + col for col in train_elevator_features.columns]

# 保存到columns目录
train_elevator_path = f"{output_dir}/columns/train_梯户比例.csv"
train_elevator_features.to_csv(train_elevator_path, index=False, encoding='utf_8_sig')
print(f"\n💾 梯户特征已保存到: {train_elevator_path}")

# 更新合并文件（修正列名引用）
def update_train_merged():
    merged_train = pd.read_csv(train_merged_path)
    elevator_features = pd.read_csv(train_elevator_path)
    final_train = pd.concat([merged_train, elevator_features], axis=1)
    
    final_path = f"{output_dir}/train_merged_v7.csv"
    final_train.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的训练集已更新: {final_path}")
    
    # 验证合并后的列名
    print("\n合并后的新增列:", [col for col in final_train.columns if col.startswith('梯户_')])
    return final_train

final_train_df = update_train_merged()

# 显示处理样例（使用正确的列名）
print("\n===== 处理结果样例 =====")
sample_cols = ['梯户比例'] + [col for col in final_train_df.columns if col.startswith('梯户_')][:3]
display(final_train_df[sample_cols].head(5))

===== 梯户比例处理验证 =====
1. 电梯数量分布:
1    84133
Name: 电梯数量, dtype: int64

2. 每梯户数描述统计:
count    84133.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: 每梯户数, dtype: float64

3. 拥挤度分布:
非常宽松    84133
舒适          0
一般          0
拥挤          0
非常拥挤        0
Name: 拥挤度, dtype: int64

💾 梯户特征已保存到: /home/mw/project/output/columns/train_梯户比例.csv
🔄 合并后的训练集已更新: /home/mw/project/output/train_merged_v7.csv

合并后的新增列: ['梯户_电梯数量', '梯户_每梯户数', '梯户_总户数', '梯户_电梯服务密度', '梯户_拥挤度', '梯户_豪宅配置', '梯户_经济型']

===== 处理结果样例 =====


梯户比例  梯户_电梯数量  梯户_每梯户数  梯户_总户数
0    一梯三户        1        1       1
1    一梯两户        1        1       1
2    一梯一户        1        1       1
3  三梯二十三户        1        1       1
4   两梯十一户        1        1       1

In [37]:
# 加载测试集合并数据
test_merged_path = f"{output_dir}/test_merged_v6.csv"
test_df = pd.read_csv(test_merged_path)

def preprocess_test_elevator_ratio(df):
    """
    测试集梯户比例预处理（与训练集完全一致）
    参数:
        df: 测试集DataFrame
    返回:
        处理后的特征DataFrame
    """
    df = df.copy()
    
    # 统一格式处理（与训练集相同）
    df['梯户比例'] = df['梯户比例'].str.replace(r'[□\s]', '', regex=True)
    
    # 相同的正则表达式提取
    pattern = r'(\d+)梯(\d+)户'
    extracted = df['梯户比例'].str.extract(pattern)
    
    # 相同的默认值处理
    df['电梯数量'] = extracted[0].fillna('1').astype(int)
    df['每梯户数'] = extracted[1].fillna('1').astype(int)
    
    # 相同的计算逻辑
    df['总户数'] = df['电梯数量'] * df['每梯户数']
    df['电梯服务密度'] = df['总户数'] / df['电梯数量']
    
    # 相同的异常值阈值
    df.loc[df['电梯数量'] > 20, '电梯数量'] = 20
    df.loc[df['每梯户数'] > 100, '每梯户数'] = 100
    
    # 相同的分箱边界
    bins = [0, 4, 8, 12, 20, 1000]
    labels = ['非常宽松', '舒适', '一般', '拥挤', '非常拥挤']
    df['拥挤度'] = pd.cut(df['电梯服务密度'], bins=bins, labels=labels)
    
    # 相同的衍生特征
    df['豪宅配置'] = (df['电梯服务密度'] <= 4).astype(int)
    df['经济型'] = (df['电梯服务密度'] >= 12).astype(int)
    
    # 相同的列名前缀处理
    features = df[['电梯数量', '每梯户数', '总户数', '电梯服务密度', '拥挤度', '豪宅配置', '经济型']]
    features.columns = ['梯户_' + col for col in features.columns]
    
    return features

# 应用预处理
test_elevator_features = preprocess_test_elevator_ratio(test_df)

# 验证处理一致性
print("===== 测试集梯户处理验证 =====")
print("1. 电梯数量分布:")
print(test_elevator_features['梯户_电梯数量'].value_counts().sort_index())

print("\n2. 服务密度分布:")
print(test_elevator_features['梯户_电梯服务密度'].describe())

print("\n3. 与训练集分箱对比:")
train_bins = pd.read_csv(f"{output_dir}/columns/train_梯户比例.csv")
print("训练集拥挤度分布:\n", train_bins['梯户_拥挤度'].value_counts())
print("\n测试集拥挤度分布:\n", test_elevator_features['梯户_拥挤度'].value_counts())

# 保存到columns目录
test_elevator_path = f"{output_dir}/columns/test_梯户比例.csv"
test_elevator_features.to_csv(test_elevator_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集梯户特征已保存到: {test_elevator_path}")

# 更新测试集合并文件
def update_test_merged():
    merged_test = pd.read_csv(test_merged_path)
    elevator_features = pd.read_csv(test_elevator_path)
    final_test = pd.concat([merged_test, elevator_features], axis=1)
    
    final_path = f"{output_dir}/test_merged_v7.csv"
    final_test.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_path}")
    
    # 列名验证
    expected_cols = ['梯户_电梯数量', '梯户_每梯户数', '梯户_总户数', 
                    '梯户_电梯服务密度', '梯户_拥挤度', '梯户_豪宅配置', '梯户_经济型']
    missing = set(expected_cols) - set(final_test.columns)
    assert len(missing) == 0, f"缺失列: {missing}"
    
    return final_test

final_test_df = update_test_merged()

# 显示处理样例
print("\n===== 测试集处理结果样例 =====")
sample_cols = ['梯户比例'] + list(test_elevator_features.columns)[:4]
display(final_test_df[sample_cols].head(5))

===== 测试集梯户处理验证 =====
1. 电梯数量分布:
1    14786
Name: 梯户_电梯数量, dtype: int64

2. 服务密度分布:
count    14786.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: 梯户_电梯服务密度, dtype: float64

3. 与训练集分箱对比:
训练集拥挤度分布:
 非常宽松    84133
Name: 梯户_拥挤度, dtype: int64

测试集拥挤度分布:
 非常宽松    14786
舒适          0
一般          0
拥挤          0
非常拥挤        0
Name: 梯户_拥挤度, dtype: int64

💾 测试集梯户特征已保存到: /home/mw/project/output/columns/test_梯户比例.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged_v7.csv

===== 测试集处理结果样例 =====


梯户比例  梯户_电梯数量  梯户_每梯户数  梯户_总户数  梯户_电梯服务密度
0  两梯两户        1        1       1        1.0
1  一梯两户        1        1       1        1.0
2  一梯三户        1        1       1        1.0
3  一梯两户        1        1       1        1.0
4  一梯两户        1        1       1        1.0

In [38]:
def preprocess_elevator(df):
    """
    配备电梯列预处理函数
    处理逻辑：
    1. 处理空白值（根据建筑高度推断）
    2. 二元编码（有=1，无=0）
    3. 创建衍生特征
    """
    df = df.copy()
    
    # 空白值处理规则（根据建筑高度推断）
    df['配备电梯'] = df['配备电梯'].fillna('未知')
    df.loc[(df['配备电梯'] == '未知') & (df['楼层_总楼层数'] >= 7), '配备电梯'] = '有'  # 7层以上建筑默认有电梯
    df.loc[(df['配备电梯'] == '未知') & (df['楼层_总楼层数'] < 7), '配备电梯'] = '无'  # 7层以下默认无
    
    # 二元编码
    df['电梯_配备'] = df['配备电梯'].apply(lambda x: 1 if x == '有' else 0)
    
    # 创建矛盾检测特征（当低楼层标注有电梯时标记）
    if '楼层_总楼层数' in df.columns:
        df['电梯_矛盾'] = ((df['电梯_配备'] == 1) & (df['楼层_总楼层数'] < 3)).astype(int)
    
    return df

# ==================== 训练集处理 ====================
# 加载之前合并的数据（确保包含楼层信息）
train_merged_path = f"{output_dir}/train_merged_v7.csv"
train_df = pd.read_csv(train_merged_path)

# 应用电梯预处理
train_elevator_processed = preprocess_elevator(train_df)

# 验证处理结果
print("===== 电梯配备处理验证 =====")
print("1. 原始分布:")
print(train_elevator_processed['配备电梯'].value_counts(dropna=False))

print("\n2. 编码后分布:")
print("有电梯占比:", f"{train_elevator_processed['电梯_配备'].mean():.1%}")

if '电梯_矛盾' in train_elevator_processed.columns:
    print("\n3. 矛盾标记（低层有电梯）:", 
          f"{train_elevator_processed['电梯_矛盾'].sum()}处")

# 保存电梯特征
elevator_cols = [col for col in train_elevator_processed.columns 
                if col.startswith('电梯_')]
train_elevator_features = train_elevator_processed[elevator_cols]

# 保存到columns目录
train_elevator_path = f"{output_dir}/columns/train_配备电梯.csv"
train_elevator_features.to_csv(train_elevator_path, index=False, encoding='utf_8_sig')
print(f"\n💾 电梯特征已保存到: {train_elevator_path}")

# 更新合并文件
def update_train_merged():
    merged_train = pd.read_csv(train_merged_path)
    elevator_features = pd.read_csv(train_elevator_path)
    final_train = pd.concat([merged_train, elevator_features], axis=1)
    
    final_path = f"{output_dir}/train_merged_v8.csv"
    final_train.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的训练集已更新: {final_path}")
    return final_train

final_train_df = update_train_merged()

# 显示处理样例
print("\n===== 处理结果样例 =====")
sample_cols = ['配备电梯', '电梯_配备']
if '电梯_矛盾' in final_train_df.columns:
    sample_cols.append('电梯_矛盾')
display(final_train_df[sample_cols].head(5))

===== 电梯配备处理验证 =====
1. 原始分布:
有    59101
无    25032
Name: 配备电梯, dtype: int64

2. 编码后分布:
有电梯占比: 70.2%

3. 矛盾标记（低层有电梯）: 59101处

💾 电梯特征已保存到: /home/mw/project/output/columns/train_配备电梯.csv
🔄 合并后的训练集已更新: /home/mw/project/output/train_merged_v8.csv

===== 处理结果样例 =====


配备电梯  电梯_配备  电梯_矛盾
0    无      0      0
1    无      0      0
2    无      0      0
3    有      1      1
4    有      1      1

In [39]:
# 加载测试集合并数据（确保包含楼层信息）
test_merged_path = f"{output_dir}/test_merged_v7.csv"
test_df = pd.read_csv(test_merged_path)

def preprocess_test_elevator(df):
    """
    测试集电梯预处理（与训练集完全一致）
    参数:
        df: 测试集DataFrame
    返回:
        处理后的特征DataFrame
    """
    df = df.copy()
    
    # 相同的空白值推断规则
    df['配备电梯'] = df['配备电梯'].fillna('未知')
    df.loc[(df['配备电梯'] == '未知') & (df['楼层_总楼层数'] >= 7), '配备电梯'] = '有'
    df.loc[(df['配备电梯'] == '未知') & (df['楼层_总楼层数'] < 7), '配备电梯'] = '无'
    
    # 相同的编码逻辑
    df['电梯_配备'] = df['配备电梯'].apply(lambda x: 1 if x == '有' else 0)
    
    # 相同的矛盾检测
    if '楼层_总楼层数' in df.columns:
        df['电梯_矛盾'] = ((df['电梯_配备'] == 1) & (df['楼层_总楼层数'] < 3)).astype(int)
    
    return df[['电梯_配备', '电梯_矛盾']] if '电梯_矛盾' in df.columns else df[['电梯_配备']]

# 应用预处理
test_elevator_features = preprocess_test_elevator(test_df)

# 验证处理一致性
print("===== 测试集电梯处理验证 =====")
print("1. 处理后分布:")
print("有电梯占比:", f"{test_elevator_features['电梯_配备'].mean():.1%}")

if '电梯_矛盾' in test_elevator_features.columns:
    print("矛盾标记数量:", test_elevator_features['电梯_矛盾'].sum())

# 对比训练集分布
train_elevator = pd.read_csv(f"{output_dir}/columns/train_配备电梯.csv")
print("\n2. 与训练集对比:")
print(f"训练集有电梯占比: {train_elevator['电梯_配备'].mean():.1%}")
print(f"测试集有电梯占比: {test_elevator_features['电梯_配备'].mean():.1%}")

# 保存到columns目录
test_elevator_path = f"{output_dir}/columns/test_配备电梯.csv"
test_elevator_features.to_csv(test_elevator_path, index=False, encoding='utf_8_sig')
print(f"\n💾 测试集电梯特征已保存到: {test_elevator_path}")

# 更新测试集合并文件
def update_test_merged():
    merged_test = pd.read_csv(test_merged_path)
    elevator_features = pd.read_csv(test_elevator_path)
    final_test = pd.concat([merged_test, elevator_features], axis=1)
    
    final_path = f"{output_dir}/test_merged_v8.csv"
    final_test.to_csv(final_path, index=False, encoding='utf_8_sig')
    print(f"🔄 合并后的测试集已更新: {final_path}")
    
    # 验证列是否存在
    assert '电梯_配备' in final_test.columns, "电梯_配备列缺失"
    return final_test

final_test_df = update_test_merged()

# 显示处理样例
print("\n===== 测试集处理结果样例 =====")
sample_cols = ['配备电梯', '电梯_配备']
if '电梯_矛盾' in final_test_df.columns:
    sample_cols.append('电梯_矛盾')
display(final_test_df[sample_cols].head(5))

===== 测试集电梯处理验证 =====
1. 处理后分布:
有电梯占比: 69.6%
矛盾标记数量: 2

2. 与训练集对比:
训练集有电梯占比: 70.2%
测试集有电梯占比: 69.6%

💾 测试集电梯特征已保存到: /home/mw/project/output/columns/test_配备电梯.csv
🔄 合并后的测试集已更新: /home/mw/project/output/test_merged_v8.csv

===== 测试集处理结果样例 =====


配备电梯  电梯_配备  电梯_矛盾
0    有      1      0
1    有      1      0
2    有      1      0
3    无      0      0
4    无      0      0

In [51]:
import pandas as pd
import numpy as np
from datetime import datetime

def process_datetime_columns(df):
    """
    处理时间相关列（交易时间和上次交易）
    保留年份信息并计算上次交易间隔
    """
    df = df.copy()
    
    # 交易时间处理
    df['交易年份'] = pd.to_datetime(df['交易时间']).dt.year
    
    # 上次交易处理（计算与当前交易的间隔年数）
    df['上次交易年份'] = pd.to_datetime(df['上次交易']).dt.year
    df['交易间隔年数'] = df['交易年份'] - df['上次交易年份']
    
    # 处理异常值
    df['交易间隔年数'] = df['交易间隔年数'].clip(lower=0, upper=100)  # 限制在0-100年之间
    
    return df[['交易年份', '上次交易年份', '交易间隔年数']]

def clean_and_process_data(df, is_train=True):
    """
    执行列删除和时间处理的主函数
    参数:
        df: 输入DataFrame
        is_train: 是否为训练集（用于差异化处理）
    """
    # 需要删除的列列表
    cols_to_drop = [
        '别墅类型', '交易权属', '房屋用途', '房屋年限', 
        '产权所属', '抵押信息', '房屋优势', '核心卖点',
        '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份'
    ]
    
    # 实际删除存在的列
    cols_to_drop = [col for col in cols_to_drop if col in df.columns]
    df = df.drop(columns=cols_to_drop)
    
    # 处理时间列
    datetime_features = process_datetime_columns(df)
    df = pd.concat([df, datetime_features], axis=1)
    
    # 移除原始时间列
    df = df.drop(columns=['交易时间', '上次交易'])
    
    return df

# ==================== 训练集处理 ====================
train_merged_path = f"{output_dir}/train_merged_v7.csv"
train_df = pd.read_csv(train_merged_path)

# 应用处理
train_processed = clean_and_process_data(train_df, is_train=True)

# 保存处理后的训练集
train_output_path = f"{output_dir}/train_merged_final.csv"
train_processed.to_csv(train_output_path, index=False, encoding='utf_8_sig')
print(f"✅ 训练集处理完成，保存到: {train_output_path}")

# ==================== 测试集处理 ====================
test_merged_path = f"{output_dir}/test_merged_v7.csv"
test_df = pd.read_csv(test_merged_path)

# 应用相同处理
test_processed = clean_and_process_data(test_df, is_train=False)

# 保存处理后的测试集
test_output_path = f"{output_dir}/test_merged_final.csv"
test_processed.to_csv(test_output_path, index=False, encoding='utf_8_sig')
print(f"✅ 测试集处理完成，保存到: {test_output_path}")

# ==================== 验证处理结果 ====================
print("\n===== 训练集处理结果验证 =====")
print("保留的列:", train_processed.columns.tolist())
print("\n交易年份分布:\n", train_processed['交易年份'].value_counts().sort_index())
print("\n交易间隔年数描述:\n", train_processed['交易间隔年数'].describe())

print("\n===== 测试集处理结果验证 =====")
print("保留的列:", test_processed.columns.tolist())
print("\n交易年份分布:\n", test_processed['交易年份'].value_counts().sort_index())

# 保存列处理日志
processing_log = {
    'removed_columns': cols_to_drop,
    'created_columns': ['交易年份', '上次交易年份', '交易间隔年数'],
    'train_shape': train_processed.shape,
    'test_shape': test_processed.shape
}

pd.DataFrame.from_dict(processing_log, orient='index').to_csv(
    f"{output_dir}/data_cleaning_log.csv", 
    header=False, 
    encoding='utf_8_sig'
)
print(f"\n📝 处理日志已保存到: {output_dir}/data_cleaning_log.csv")

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 3018-03-26 00:00:00